<a href="https://colab.research.google.com/github/abhi-shek-solanki/zeotap/blob/main/Abhishek_Solanki_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **TASK -2**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Importing the data

df_customers = pd.read_csv('/content/project/Customers.csv')
df_products = pd.read_csv('/content/project/Products.csv')
df_transaction = pd.read_csv('/content/project/Transactions.csv')


In [ ]:
# cheking the data for outliers

df_customers.info()
df_products.info()
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    200 non-null    object
 1   CustomerName  200 non-null    object
 2   Region        200 non-null    object
 3   SignupDate    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    100 non-null    object 
 1   ProductName  100 non-null    object 
 2   Category     100 non-null    object 
 3   Price        100 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  


In [ ]:
df_customers.isnull().sum()
df_products.isnull().sum()
df_transaction.isnull().sum()

,0
TransactionID,0
CustomerID,0
ProductID,0
TransactionDate,0
Quantity,0
TotalValue,0
Price,0


our data is clear lets applying a lookalike model

In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# merging product and customer table

df_data = pd.merge(df_customers, df_transaction, on='CustomerID')
df_data = pd.merge(df_data, df_products, on='ProductID')



In [ ]:
# Now our new data is df_data

df_data.info()

# now I  want to drop some colums which are useless

df_data.drop(['TransactionID', 'CustomerID', 'CustomerName','SignupDate','TransactionDate','Quantity','TotalValue','Price_x','Price_y'], axis=1, inplace=True)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerID       1000 non-null   object 
 1   CustomerName     1000 non-null   object 
 2   Region           1000 non-null   object 
 3   SignupDate       1000 non-null   object 
 4   TransactionID    1000 non-null   object 
 5   ProductID        1000 non-null   object 
 6   TransactionDate  1000 non-null   object 
 7   Quantity         1000 non-null   int64  
 8   TotalValue       1000 non-null   float64
 9   Price_x          1000 non-null   float64
 10  ProductName      1000 non-null   object 
 11  Category         1000 non-null   object 
 12  Price_y          1000 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 101.7+ KB


In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Region       1000 non-null   object
 1   ProductID    1000 non-null   object
 2   ProductName  1000 non-null   object
 3   Category     1000 non-null   object
dtypes: object(4)
memory usage: 31.4+ KB


In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load the Customers Data
df = df_data

# Step 2: Filter the first 20 customers (CustomerID: C0001 - C0020)
df_filtered = df[df["CustomerID"].isin([f"C{str(i).zfill(4)}" for i in range(1, 21)])]

# Step 3: Select Features for Similarity Calculation
features = ['Price_x','Price_y', 'Category', 'Region']  # Modify as needed

# Encode Categorical Variables
encoder = LabelEncoder()
df_filtered['Region'] = encoder.fit_transform(df_filtered['Region'])
df_filtered['Category'] = encoder.fit_transform(df_filtered['Category'])

# Extract numerical features
customer_features = df_filtered[features]

# Step 4: Normalize Data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features)

# Step 5: Compute Cosine Similarity Matrix
similarity_matrix = cosine_similarity(scaled_features)

# Step 6: Generate Lookalike Mapping
lookalike_map = {}

for idx, cust_id in enumerate(df_filtered['CustomerID']):
    # Get similarity scores for the current customer
    similarity_scores = list(enumerate(similarity_matrix[idx]))

    # Sort by similarity (excluding itself)
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:4]

    # Get top 3 similar customers and their scores
    similar_customers = [(df_filtered.iloc[i]['CustomerID'], round(score, 4)) for i, score in similarity_scores]

    # Store in dictionary
    lookalike_map[cust_id] = similar_customers

# Step 7: Save Results as Lookalike.csv
lookalike_df = pd.DataFrame(list(lookalike_map.items()), columns=['CustomerID', 'Lookalikes'])
lookalike_df.to_csv("Lookalike.csv", index=False)

# Step 8: Print Sample Output
print("Lookalike Mapping (Top 3 Similar Customers with Scores):")
print(lookalike_df.head(3))


Lookalike Mapping (Top 3 Similar Customers with Scores):
  CustomerID                                         Lookalikes
0      C0001  [(C0001, 0.9986), (C0012, 0.9985), (C0013, 0.9...
1      C0002  [(C0009, 0.9679), (C0005, 0.922), (C0010, 0.88...
2      C0003  [(C0013, 0.9853), (C0001, 0.961), (C0001, 0.93...


<ipython-input-23-f17e79b0e1ef>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Region'] = encoder.fit_transform(df_filtered['Region'])
<ipython-input-23-f17e79b0e1ef>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Category'] = encoder.fit_transform(df_filtered['Category'])


In [ ]:
lookalike_df


,CustomerID,Lookalikes
0,C0001,"[(C0001, 0.9986), (C0012, 0.9985), (C0013, 0.9..."
1,C0002,"[(C0009, 0.9679), (C0005, 0.922), (C0010, 0.88..."
2,C0003,"[(C0013, 0.9853), (C0001, 0.961), (C0001, 0.93..."
3,C0004,"[(C0001, 0.9249), (C0011, 0.917), (C0004, 0.86..."
4,C0005,"[(C0019, 0.9716), (C0017, 0.951), (C0002, 0.94..."
5,C0006,"[(C0013, 1.0), (C0013, 0.995), (C0011, 0.9914)]"
6,C0007,"[(C0005, 0.9959), (C0017, 0.9594), (C0002, 0.9..."
7,C0008,"[(C0008, 0.9488), (C0013, 0.9395), (C0008, 0.9..."
8,C0009,"[(C0017, 1.0), (C0002, 0.9581), (C0010, 0.8967)]"
9,C0010,"[(C0010, 0.9976), (C0009, 0.9418), (C0019, 0.9..."
